# Notebook Details

In this notebook we will give Long term memory.

We have 2 tools:
- 1. Calculator Tools.
- 2. wikipwdia web search.

We will store old conversion to a list and pass this histoy as context of the query

## Defining Tools

In [1]:
# import Necessary Library
import requests
import ollama
import json
import os

In [2]:
# --- Configuration --- #
MODEL_NAMAE = "llama3.1"
MEMORY_PATH = "memory.json"

In [3]:
# 1. Simple calculator tool
def calculator_tool(query):
    """A simple calculator tool."""
    try:
        # Keep only numbers, operators, parentheses, and dots
        cleaned_expr = query.split('=')[0]  # Remove anything after '='
        cleaned_expr = "".join(cleaned_expr).strip()
        return str(eval(cleaned_expr))
    except Exception as e:
        return f"Error in calculation: {e}"

In [4]:
# 2. Wikipedia search tool
def web_search(query):
    url = "https://en.wikipedia.org/w/api.php"
    params = {
        "action": "query",
        "list": "search",
        "srsearch": query,
        "format": "json"
    }
    try:
        response = requests.get(url, params=params)
        data = response.json()
        if "query" in data and "search" in data["query"]:
            results = [item['snippet'] for item in data['query']['search']]
            return "\n".join(results) if results else "No results found on Wikipedia."
        else:
            return "No results found."
    except Exception as e:
        return f"Error searching Wikipedia: {e}"

## Defining Memory

In [5]:
# Function to load memory from a file
def load_memory():
    if os.path.exists(MEMORY_PATH):
        with open(MEMORY_PATH, "r") as f:
            return json.load(f)
    else:
        # Start with a fresh system prompt if no memory file exists
        return [{
            "role": "system", 
            "content": """
            You are a helpful AI assistant. You can either perform calculations or search the web.
            - CALCULATE: <math expression>
            - SEARCH: <search query>
            - ANSWER: <your answer>

            Output ONLY one of the above actions on the FIRST line. 
            Do not explain your reasoning. Do not add extra commentary.

            Always use the information from previous conversation if relevant.
            Output ONLY one of the above actions on the FIRST line.
            Do not explain your reasoning. Do not add extra commentary.
           
            Example:
            Q: What is 12*(5+2)?
            A: CALCULATE: 12*(5+2)
            
            Q: Who is the president of the USA?
            A: SEARCH: Who is the president of the USA?
            
            Q: Tell me a joke about Cats.
            A: ANSWER: Sure, here is a joke...
            """}]

# Function to save memory to a file
def save_memory(memory):
    with open(MEMORY_PATH, "w") as f:
        json.dump(memory, f, indent=4)

In [6]:
MEMORY_PATH

'memory.json'

## Defining Agent

In [7]:
# Loading memory
memory = load_memory()

In [8]:
def agent(query, model="llama3.2:latest"):
    # Defining the global memory variable
    global memory

    # Adding user query to memory
    memory.append({"role": "user", "content": query})

    # Ask LLM with conversation history
    decision_response = ollama.chat(
        model=model,
        messages=memory
    )

    # Extracting the tools name from decision response
    decision = decision_response['message']['content'].strip()

    # save model's reasoning memory
    memory.append({"role": "assistant", "content": decision})

    # Get  first line of (action)
    first_line = decision.split('\n')[0].strip()

    print(f"\tDecision made by LLM: {first_line}")

    # Exectuing the tools
    if "CALCULATE" in first_line:
        expression = first_line.split("CALCULATE:", 1)[1].strip()
        result = calculator_tool(expression)
    elif "SEARCH:" in first_line:
        search_query = first_line.split("SEARCH:")[1].strip()
        result = web_search(search_query)
    elif "ANSWER:" in first_line:
        result = first_line.split("ANSWER:")[1].strip()
    else:
        result = "Invalid action specified."
    
    # Savinig Result to memory
    memory.append({"role": "assistant", "content": result})

    # Save to file
    with open("memory.json", "w") as f:
        json.dump(memory, f)

    return result


In [10]:
agent("How many country is in Europe?")

	Decision made by LLM: ANSWER: 50


'50'

In [12]:
agent("Who is the ceo of Google?")

	Decision made by LLM: ANSWER: Sundar Pichai


'Sundar Pichai'

In [10]:
agent("How old is he?")

	Decision made by LLM: ANSWER: 51


'51'

In [11]:
agent("From which institution did he graduate?")

	Decision made by LLM: ANSWER: Stanford University


'Stanford University'